In [1]:
import os
from typing import Optional


def set_project_root_dir(project_root_name: str, cwd: Optional[str] = None):
    """Set the working directory to the project root directory, based on the name of the project root directory.

    Args:
        project_root_name (str): The name of the project root directory.
        cwd (str, optional): The current working directory. Defaults to None.

    Raises:
        ValueError: If the project root directory is not found in the directory hierarchy.

    Returns:
        None
    """
    # If no current working directory is provided, use the current working directory
    if cwd is None:
        cwd = os.getcwd()

    # Split the current working directory into its components
    cwd_components = cwd.split(os.sep)

    # Find the index of the first occurrence of the project root directory in the list of components
    try:
        root_index = cwd_components.index(project_root_name)
    except ValueError:
        raise ValueError(
            f"Project root directory '{project_root_name}' not found in directory hierarchy."
        )

    # Use the root index to get the path of the project root directory
    root_dir = os.sep.join(cwd_components[: root_index + 1])

    # Change the working directory to the project root directory
    os.chdir(root_dir)

    # Print new CWD
    print("New CWD is: " + os.getcwd())


set_project_root_dir("backtestbuddy")

New CWD is: i:\Coding\00_Projects\00_packages\backtestbuddy


In [2]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.model_selection import TimeSeriesSplit # type: ignore
from sklearn.pipeline import make_pipeline # type: ignore
from sklearn.preprocessing import StandardScaler # type: ignore

from backtestbuddy.backtest.sport_backtest import ModelBacktest, PredictionBacktest
from backtestbuddy.strategies.sport_strategies import FixedStake, KellyCriterion

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
import backtestbuddy
backtestbuddy.__version__

'0.1.1'

In [4]:
# Cell 2: Create dummy data
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
n_samples = len(dates)

data = pd.DataFrame({
    'date': dates,
    'odds_team_a': np.random.uniform(1.5, 3, n_samples),
    'odds_team_b': np.random.uniform(1.5, 3, n_samples),
    'actual_winner': np.random.randint(0, 2, n_samples),
    'model_predictions': np.random.randint(0, 2, n_samples)
})

# Cell 3: Display the first few rows of the dummy data
data

,date,odds_team_a,odds_team_b,actual_winner,model_predictions
0,2023-01-01,2.061810,2.079154,0,0
1,2023-01-02,2.926071,2.941786,0,1
2,2023-01-03,2.597991,2.858026,0,1
3,2023-01-04,2.397988,1.793687,0,1
4,2023-01-05,1.734028,1.604042,0,0
...,...,...,...,...,...
360,2023-12-27,2.082255,2.762743,0,0
361,2023-12-28,2.464932,1.709659,0,0
362,2023-12-29,2.187379,2.692901,1,1
363,2023-12-30,2.318425,1.802441,1,1


In [5]:
# Cell 4: Set up the backtester
strategy = FixedStake(stake=10)
backtest = PredictionBacktest(
    data=data,
    date_column='date',
    odds_columns=['odds_team_a', 'odds_team_b'],
    outcome_column='actual_winner',
    prediction_column='model_predictions',
    initial_bankroll=1000,
    strategy=strategy,
)

In [6]:
backtest.run()

In [7]:
detailed_results = backtest.detailed_results
detailed_results

,bt_index,bt_fold,bt_predicted_outcome,bt_bet_on,bt_actual_outcome,bt_starting_bankroll,bt_ending_bankroll,bt_odds,bt_win,bt_profit,bt_roi,bt_stake,bt_potential_return,bt_date_column,bt_odd_0,bt_odd_1,date,odds_team_a,odds_team_b,actual_winner,model_predictions
0,0,0,0,0,0,1000.000000,1010.618102,2.061810,True,10.618102,106.181018,10,20.618102,2023-01-01,2.061810,2.079154,2023-01-01,2.061810,2.079154,0,0
1,1,0,1,1,0,1010.618102,1000.618102,2.941786,False,-10.000000,-100.000000,10,29.417858,2023-01-02,2.926071,2.941786,2023-01-02,2.926071,2.941786,0,1
2,2,0,1,1,0,1000.618102,990.618102,2.858026,False,-10.000000,-100.000000,10,28.580260,2023-01-03,2.597991,2.858026,2023-01-03,2.597991,2.858026,0,1
3,3,0,1,1,0,990.618102,980.618102,1.793687,False,-10.000000,-100.000000,10,17.936867,2023-01-04,2.397988,1.793687,2023-01-04,2.397988,1.793687,0,1
4,4,0,0,0,0,980.618102,987.958381,1.734028,True,7.340280,73.402796,10,17.340280,2023-01-05,1.734028,1.604042,2023-01-05,1.734028,1.604042,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,0,0,0,0,1236.142029,1246.964578,2.082255,True,10.822549,108.225489,10,20.822549,2023-12-27,2.082255,2.762743,2023-12-27,2.082255,2.762743,0,0
361,361,0,0,0,0,1246.964578,1261.613901,2.464932,True,14.649323,146.493233,10,24.649323,2023-12-28,2.464932,1.709659,2023-12-28,2.464932,1.709659,0,0
362,362,0,1,1,1,1261.613901,1278.542911,2.692901,True,16.929010,169.290097,10,26.929010,2023-12-29,2.187379,2.692901,2023-12-29,2.187379,2.692901,1,1
363,363,0,1,1,1,1278.542911,1286.567320,1.802441,True,8.024410,80.244098,10,18.024410,2023-12-30,2.318425,1.802441,2023-12-30,2.318425,1.802441,1,1


In [8]:
bookie_results = backtest.get_detailed_results()
bookie_results

,bt_index,bt_fold,bt_predicted_outcome,bt_bet_on,bt_actual_outcome,bt_starting_bankroll,bt_ending_bankroll,bt_odds,bt_win,bt_profit,bt_roi,bt_stake,bt_potential_return,bt_date_column,bt_odd_0,bt_odd_1,date,odds_team_a,odds_team_b,actual_winner,model_predictions
0,0,0,0,0,0,1000.000000,1010.618102,2.061810,True,10.618102,106.181018,10,20.618102,2023-01-01,2.061810,2.079154,2023-01-01,2.061810,2.079154,0,0
1,1,0,1,1,0,1010.618102,1000.618102,2.941786,False,-10.000000,-100.000000,10,29.417858,2023-01-02,2.926071,2.941786,2023-01-02,2.926071,2.941786,0,1
2,2,0,1,1,0,1000.618102,990.618102,2.858026,False,-10.000000,-100.000000,10,28.580260,2023-01-03,2.597991,2.858026,2023-01-03,2.597991,2.858026,0,1
3,3,0,1,1,0,990.618102,980.618102,1.793687,False,-10.000000,-100.000000,10,17.936867,2023-01-04,2.397988,1.793687,2023-01-04,2.397988,1.793687,0,1
4,4,0,0,0,0,980.618102,987.958381,1.734028,True,7.340280,73.402796,10,17.340280,2023-01-05,1.734028,1.604042,2023-01-05,1.734028,1.604042,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,0,0,0,0,1236.142029,1246.964578,2.082255,True,10.822549,108.225489,10,20.822549,2023-12-27,2.082255,2.762743,2023-12-27,2.082255,2.762743,0,0
361,361,0,0,0,0,1246.964578,1261.613901,2.464932,True,14.649323,146.493233,10,24.649323,2023-12-28,2.464932,1.709659,2023-12-28,2.464932,1.709659,0,0
362,362,0,1,1,1,1261.613901,1278.542911,2.692901,True,16.929010,169.290097,10,26.929010,2023-12-29,2.187379,2.692901,2023-12-29,2.187379,2.692901,1,1
363,363,0,1,1,1,1278.542911,1286.567320,1.802441,True,8.024410,80.244098,10,18.024410,2023-12-30,2.318425,1.802441,2023-12-30,2.318425,1.802441,1,1


In [9]:
bookie_results = backtest.get_bookie_results()
bookie_results

,bt_index,bt_fold,bt_predicted_outcome,bt_bet_on,bt_actual_outcome,bt_starting_bankroll,bt_ending_bankroll,bt_stake,bt_potential_return,bt_win,bt_profit,bt_roi,bt_odds,bt_date_column,bt_odd_0,bt_odd_1,date,odds_team_a,odds_team_b,actual_winner,model_predictions
0,0,0,0,0,0,1000.000000,1010.618102,10,20.618102,True,10.618102,106.181018,2.061810,2023-01-01,2.061810,2.079154,2023-01-01,2.061810,2.079154,0,0
1,1,0,0,0,0,1010.618102,1029.878816,10,29.260715,True,19.260715,192.607146,2.926071,2023-01-02,2.926071,2.941786,2023-01-02,2.926071,2.941786,0,1
2,2,0,0,0,0,1029.878816,1045.858726,10,25.979909,True,15.979909,159.799091,2.597991,2023-01-03,2.597991,2.858026,2023-01-03,2.597991,2.858026,0,1
3,3,0,1,1,0,1045.858726,1035.858726,10,17.936867,False,-10.000000,-100.000000,1.793687,2023-01-04,2.397988,1.793687,2023-01-04,2.397988,1.793687,0,1
4,4,0,1,1,0,1035.858726,1025.858726,10,16.040420,False,-10.000000,-100.000000,1.604042,2023-01-05,1.734028,1.604042,2023-01-05,1.734028,1.604042,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,360,0,0,0,0,1079.234754,1090.057303,10,20.822549,True,10.822549,108.225489,2.082255,2023-12-27,2.082255,2.762743,2023-12-27,2.082255,2.762743,0,0
361,361,0,1,1,0,1090.057303,1080.057303,10,17.096586,False,-10.000000,-100.000000,1.709659,2023-12-28,2.464932,1.709659,2023-12-28,2.464932,1.709659,0,0
362,362,0,0,0,1,1080.057303,1070.057303,10,21.873793,False,-10.000000,-100.000000,2.187379,2023-12-29,2.187379,2.692901,2023-12-29,2.187379,2.692901,1,1
363,363,0,1,1,1,1070.057303,1078.081713,10,18.024410,True,8.024410,80.244098,1.802441,2023-12-30,2.318425,1.802441,2023-12-30,2.318425,1.802441,1,1


In [10]:
backtest.calculate_metrics()

{'Backtest Start Date': Timestamp('2023-01-01 00:00:00'),
 'Backtest End Date': Timestamp('2023-12-31 00:00:00'),
 'Backtest Duration': Timedelta('364 days 00:00:00'),
 'ROI [%]': np.float64(30.56892924637293),
 'Total Profit [$]': np.float64(305.6892924637296),
 'Bankroll Final [$]': np.float64(1305.6892924637293),
 'Bankroll Peak [$]': np.float64(1305.6892924637293),
 'Bankroll Valley [$]': np.float64(875.1124844618936),
 'Sharpe Ratio [-]': np.float64(1.1282080939737171),
 'Sortino Ratio [-]': np.float64(15.701799445019583),
 'Calmar Ratio [-]': np.float64(1.3003556137214456),
 'Max Drawdown [%]': np.float64(15.366568563360786),
 'Max. Drawdown Duration [bets]': np.int64(82),
 'Win Rate [%]': np.float64(48.76712328767123),
 'Average Odds [-]': np.float64(2.2156884264822603),
 'Highest Winning Odds [-]': np.float64(2.9894471941789504),
 'Highest Losing Odds [-]': np.float64(2.980330404900776),
 'Average Stake [$]': np.float64(10.0),
 'Best Bet [$]': np.float64(19.894471941789504),
 '

In [11]:
backtest.metrics

{'Backtest Start Date': Timestamp('2023-01-01 00:00:00'),
 'Backtest End Date': Timestamp('2023-12-31 00:00:00'),
 'Backtest Duration': Timedelta('364 days 00:00:00'),
 'ROI [%]': np.float64(30.56892924637293),
 'Total Profit [$]': np.float64(305.6892924637296),
 'Bankroll Final [$]': np.float64(1305.6892924637293),
 'Bankroll Peak [$]': np.float64(1305.6892924637293),
 'Bankroll Valley [$]': np.float64(875.1124844618936),
 'Sharpe Ratio [-]': np.float64(1.1282080939737171),
 'Sortino Ratio [-]': np.float64(15.701799445019583),
 'Calmar Ratio [-]': np.float64(1.3003556137214456),
 'Max Drawdown [%]': np.float64(15.366568563360786),
 'Max. Drawdown Duration [bets]': np.int64(82),
 'Win Rate [%]': np.float64(48.76712328767123),
 'Average Odds [-]': np.float64(2.2156884264822603),
 'Highest Winning Odds [-]': np.float64(2.9894471941789504),
 'Highest Losing Odds [-]': np.float64(2.980330404900776),
 'Average Stake [$]': np.float64(10.0),
 'Best Bet [$]': np.float64(19.894471941789504),
 '

In [12]:
backtest.plot()

In [13]:
backtest.plot_odds_distribution()


In [14]:
model = make_pipeline(StandardScaler(), LogisticRegression())

# Initialize ModelBacktest with the dummy model
model_backtest = ModelBacktest(
    data=data,
    date_column='date',
    odds_columns=['odds_team_a', 'odds_team_b'],
    outcome_column='actual_winner',
    initial_bankroll=1000,
    strategy=strategy,
    cv_schema=TimeSeriesSplit(n_splits=5),
    model=model
)


In [15]:
# Run the backtest
model_backtest.run()

In [16]:
model_backtest.get_bookie_results()
model_backtest.detailed_results

,bt_index,bt_fold,bt_predicted_outcome,bt_bet_on,bt_actual_outcome,bt_starting_bankroll,bt_ending_bankroll,bt_odds,bt_win,bt_profit,bt_roi,bt_stake,bt_potential_return,bt_date_column,bt_odd_0,bt_odd_1,bt_model_prob_0,bt_model_prob_1,date,odds_team_a,odds_team_b,actual_winner,model_predictions
0,65,0,0,0,0,1000.000000,1013.140441,2.314044,True,13.140441,131.404412,10,23.140441,2023-03-07,2.314044,1.714488,0.698902,0.301098,2023-03-07,2.314044,1.714488,0,1
1,66,0,1,1,0,1013.140441,1003.140441,2.642266,False,-10.000000,-100.000000,10,26.422659,2023-03-08,1.711386,2.642266,0.499435,0.500565,2023-03-08,1.711386,2.642266,0,1
2,67,0,0,0,0,1003.140441,1020.173396,2.703295,True,17.032955,170.329547,10,27.032955,2023-03-09,2.703295,2.427327,0.634992,0.365008,2023-03-09,2.703295,2.427327,0,0
3,68,0,0,0,1,1020.173396,1010.173396,1.611826,False,-10.000000,-100.000000,10,16.118260,2023-03-10,1.611826,1.651684,0.584585,0.415415,2023-03-10,1.611826,1.651684,1,0
4,69,0,0,0,1,1010.173396,1000.173396,2.980330,False,-10.000000,-100.000000,10,29.803304,2023-03-11,2.980330,1.626160,0.758186,0.241814,2023-03-11,2.980330,1.626160,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,360,4,0,0,0,1384.902240,1395.724789,2.082255,True,10.822549,108.225489,10,20.822549,2023-12-27,2.082255,2.762743,0.551387,0.448613,2023-12-27,2.082255,2.762743,0,0
296,361,4,1,1,0,1395.724789,1385.724789,1.709659,False,-10.000000,-100.000000,10,17.096586,2023-12-28,2.464932,1.709659,0.494482,0.505518,2023-12-28,2.464932,1.709659,0,0
297,362,4,0,0,1,1385.724789,1375.724789,2.187379,False,-10.000000,-100.000000,10,21.873793,2023-12-29,2.187379,2.692901,0.587897,0.412103,2023-12-29,2.187379,2.692901,1,1
298,363,4,0,0,1,1375.724789,1365.724789,2.318425,False,-10.000000,-100.000000,10,23.184252,2023-12-30,2.318425,1.802441,0.544288,0.455712,2023-12-30,2.318425,1.802441,1,1


In [17]:
model_backtest.calculate_metrics()

{'Backtest Start Date': Timestamp('2023-03-07 00:00:00'),
 'Backtest End Date': Timestamp('2023-12-31 00:00:00'),
 'Backtest Duration': Timedelta('299 days 00:00:00'),
 'ROI [%]': np.float64(35.57247886718092),
 'Total Profit [$]': np.float64(355.72478867180985),
 'Bankroll Final [$]': np.float64(1355.7247886718092),
 'Bankroll Peak [$]': np.float64(1463.722726368339),
 'Bankroll Valley [$]': np.float64(980.1733959586843),
 'Sharpe Ratio [-]': np.float64(1.7791357673650765),
 'Sortino Ratio [-]': np.float64(16.72765213888011),
 'Calmar Ratio [-]': np.float64(2.8150722427794648),
 'Max Drawdown [%]': np.float64(9.483353619172444),
 'Max. Drawdown Duration [bets]': np.int64(24),
 'Win Rate [%]': np.float64(50.33333333333333),
 'Average Odds [-]': np.float64(2.2349962234426966),
 'Highest Winning Odds [-]': np.float64(2.9894471941789504),
 'Highest Losing Odds [-]': np.float64(2.999576509929196),
 'Average Stake [$]': np.float64(10.0),
 'Best Bet [$]': np.float64(19.894471941789504),
 'Wo

In [18]:
model_backtest.plot()

In [19]:
model_backtest.plot_odds_distribution()

In [20]:
# Cell 4: Set up the backtester
strategy = KellyCriterion()
model = make_pipeline(StandardScaler(), LogisticRegression())

# Initialize ModelBacktest with the dummy model
model_backtest = ModelBacktest(
    data=data,
    date_column='date',
    odds_columns=['odds_team_a', 'odds_team_b'],
    outcome_column='actual_winner',
    initial_bankroll=1000,
    strategy=strategy,
    cv_schema=TimeSeriesSplit(n_splits=5),
    model=model
)
# Run the backtest
model_backtest.run()

In [21]:
model_backtest.get_bookie_results()
model_backtest.detailed_results

,bt_index,bt_fold,bt_predicted_outcome,bt_bet_on,bt_actual_outcome,bt_starting_bankroll,bt_ending_bankroll,bt_odds,bt_win,bt_profit,bt_roi,bt_stake,bt_potential_return,bt_date_column,bt_odd_0,bt_odd_1,bt_model_prob_0,bt_model_prob_1,bt_kelly_fraction_0,bt_kelly_fraction_1,date,odds_team_a,odds_team_b,actual_winner,model_predictions
0,65,0,0,0,0,1000.000000,1131.404412,2.314044,True,131.404412,131.404412,100.000000,231.404412,2023-03-07,2.314044,1.714488,0.698902,0.301098,0.469764,0.000000,2023-03-07,2.314044,1.714488,0,1
1,66,0,1,1,0,1131.404412,1020.271350,2.642266,False,-111.133063,-100.000000,111.133063,293.643108,2023-03-08,1.711386,2.642266,0.499435,0.500565,0.000000,0.196452,2023-03-08,1.711386,2.642266,0,1
2,67,0,0,0,0,1020.271350,1194.053707,2.703295,True,173.782357,170.329547,102.027135,275.809492,2023-03-09,2.703295,2.427327,0.634992,0.365008,0.420696,0.000000,2023-03-09,2.703295,2.427327,0,0
3,68,0,0,-1,1,1194.053707,1194.053707,NaN,None,0.000000,0.000000,0.000000,0.000000,2023-03-10,1.611826,1.651684,0.584585,0.415415,0.000000,0.000000,2023-03-10,1.611826,1.651684,1,0
4,69,0,0,0,1,1194.053707,1074.648336,2.980330,False,-119.405371,-100.000000,119.405371,355.867457,2023-03-11,2.980330,1.626160,0.758186,0.241814,0.636078,0.000000,2023-03-11,2.980330,1.626160,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,360,4,0,0,0,23513.708637,25255.219466,2.082255,True,1741.510829,108.225489,1609.150345,3350.661174,2023-12-27,2.082255,2.762743,0.551387,0.448613,0.136869,0.135813,2023-12-27,2.082255,2.762743,0,0
296,361,4,1,0,0,25255.219466,28018.951551,2.464932,True,2763.732085,146.493233,1886.593690,4650.325775,2023-12-28,2.464932,1.709659,0.494482,0.505518,0.149402,0.000000,2023-12-28,2.464932,1.709659,0,0
297,362,4,0,0,1,28018.951551,25217.056396,2.187379,False,-2801.895155,-100.000000,2801.895155,6128.807563,2023-12-29,2.187379,2.692901,0.587897,0.412103,0.240827,0.064832,2023-12-29,2.187379,2.692901,1,1
298,363,4,0,0,1,25217.056396,22712.518008,2.318425,False,-2504.538388,-100.000000,2504.538388,5806.584873,2023-12-30,2.318425,1.802441,0.544288,0.455712,0.198638,0.000000,2023-12-30,2.318425,1.802441,1,1


In [22]:
model_backtest.calculate_metrics()

{'Backtest Start Date': Timestamp('2023-03-07 00:00:00'),
 'Backtest End Date': Timestamp('2023-12-31 00:00:00'),
 'Backtest Duration': Timedelta('299 days 00:00:00'),
 'ROI [%]': np.float64(2605.5599371708777),
 'Total Profit [$]': np.float64(26055.599371708773),
 'Bankroll Final [$]': np.float64(27055.599371708777),
 'Bankroll Peak [$]': np.float64(28018.951550722377),
 'Bankroll Valley [$]': np.float64(870.4651520767808),
 'Sharpe Ratio [-]': np.float64(2.4399622527496097),
 'Sortino Ratio [-]': np.float64(10.636453046521737),
 'Calmar Ratio [-]': np.float64(5.750320913343403),
 'Max Drawdown [%]': np.float64(73.91477374771705),
 'Max. Drawdown Duration [bets]': np.int64(51),
 'Win Rate [%]': 49.08424908424908,
 'Average Odds [-]': np.float64(2.501414572717206),
 'Highest Winning Odds [-]': np.float64(2.9894471941789504),
 'Highest Losing Odds [-]': np.float64(2.999576509929196),
 'Average Stake [$]': np.float64(749.4753621813271),
 'Best Bet [$]': np.float64(4343.081363856936),
 'W

In [23]:
model_backtest.plot()

In [24]:
model_backtest.plot_odds_distribution()